### Load Pthon libraries 

In [2]:
import pandas as pd 
from deep_translator import GoogleTranslator
import os 
import langid

### Load Data

In [3]:
# Create a directory named 'data' if it doesn't exist
os.makedirs('data', exist_ok=True)
# Download the CSV file into the 'data' folder
!wget -O data/incidents_train.csv https://raw.githubusercontent.com/food-hazard-detection-semeval-2025/food-hazard-detection-semeval-2025.github.io/main/data/incidents_train.csv
# Load the dataset from the 'data' folder
df_food_trial = pd.read_csv('data/incidents_train.csv', index_col=0)
# Display a sample of the dataset
df_food_trial.sample()

--2024-11-21 00:40:20--  https://raw.githubusercontent.com/food-hazard-detection-semeval-2025/food-hazard-detection-semeval-2025.github.io/main/data/incidents_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... failed: Network is unreachable.
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12866710 (12M) [text/plain]
Saving to: ‘data/incidents_train.csv’

data/incidents_trai 100%[===================>]  12,27M  2,52MB/s    in 5,2s    

2024-11-21 00:40:53 (2,34 MB/s) - ‘data/incidents_train.csv’ saved [12866710/12866710]



,year,month,day,country,title,text,hazard-category,product-category,hazard,product
4984,2021,5,4,us,Revised Guan’s Mushroom Co Recalls Enoki Becau...,"Guan’s Mushroom Co of Commerce, CA is recallin...",biological,fruits and vegetables,listeria monocytogenes,fresh mushrooms


- The code detects the language of the text in the `text` and `title` columns of the df_food_trial DataFrame and stores the language codes in new columns `language_text` and `language_title`.

In [4]:
# Detect language for each text entry in the 'text' column
df_food_trial['language_text'] = df_food_trial['text'].apply(lambda x: langid.classify(x)[0])
df_food_trial['language_title'] = df_food_trial['title'].apply(lambda x: langid.classify(x)[0])
df_food_trial

,year,month,day,country,title,text,hazard-category,product-category,hazard,product,language_text,language_title
0,1994,1,7,us,Recall Notification: FSIS-024-94,Case Number: 024-94 \n Date Opene...,biological,"meat, egg and dairy products",listeria monocytogenes,smoked sausage,en,en
1,1994,3,10,us,Recall Notification: FSIS-033-94,Case Number: 033-94 \n Date Opene...,biological,"meat, egg and dairy products",listeria spp,sausage,en,en
2,1994,3,28,us,Recall Notification: FSIS-014-94,Case Number: 014-94 \n Date Opene...,biological,"meat, egg and dairy products",listeria monocytogenes,ham slices,en,en
3,1994,4,3,us,Recall Notification: FSIS-009-94,Case Number: 009-94 \n Date Opene...,foreign bodies,"meat, egg and dairy products",plastic fragment,thermal processed pork meat,en,en
4,1994,7,1,us,Recall Notification: FSIS-001-94,Case Number: 001-94 \n Date Opene...,foreign bodies,"meat, egg and dairy products",plastic fragment,chicken breast,en,en
...,...,...,...,...,...,...,...,...,...,...,...,...
5979,2022,7,28,hk,Imported biscuit may contain allergen (peanuts),Imported biscuit may contain allergen (peanuts...,allergens,cereals and bakery products,peanuts and products thereof,biscuits,en,en
5980,2022,7,28,us,Danny’s Sub and Pizza Recalls Meat Pizza Produ...,023-2022\n\n \n High - Class I\n\n Produc...,fraud,prepared dishes and snacks,inspection issues,pizza,en,en
5981,2022,7,29,us,Lyons Magnus Voluntarily Recalls 53 Nutritiona...,"FRESNO, Calif. – July 28, 2022 – Lyons Magnus ...",biological,non-alcoholic beverages,cronobacter spp,non-alcoholic beverages,en,en
5982,2022,7,30,us,"Conagra Brands, Inc., Recalls Frozen Beef Prod...",025-2022\n\n \n High - Class I\n\n Misbra...,allergens,"meat, egg and dairy products",eggs and products thereof,frozen beef products,en,en


- In the following code, we will implement the related code for translating non english text data to the engish language.

In [5]:
# Function to translate text
def translate_text_ensure(text):
    """
    Ensures text is translated correctly.
    - Handles missing or empty strings.
    - Reattempts translation if the output is the same as the input.
    """
    if not text or pd.isnull(text):  # Skip None or empty values
        return ""
    try:
        translator = GoogleTranslator(source='auto', target='en')
        translated = translator.translate(text)
        
        # Reattempt translation if output matches input
        if translated.strip().lower() == text.strip().lower():
            translated = translator.translate(text)  # Retry translation
        
        return translated
    except Exception as e:
        return f"Error: {str(e)}"

In [6]:
print("Number of non-english for title: ", df_food_trial[df_food_trial['language_title'] != 'en'].shape[0])
print("Number of non-english for text: ", df_food_trial[df_food_trial['language_text'] != 'en'].shape[0])

Number of non-english for title:  360
Number of non-english for text:  3


In [7]:
df_non_en_text = df_food_trial[df_food_trial['language_text'] != 'en'] # 3 rows; save rows having in column "text" language diffrent than the english one. 
df_non_en_tilte = df_food_trial[df_food_trial['language_title'] != 'en'] # 360 rows;  save rows having in column "title" language diffrent than the english one. 
# Apply the translation function to the DataFrame
df_non_en_text["text_translated"] = df_non_en_text["text"].apply(translate_text_ensure)
df_non_en_tilte["title_translated"] = df_non_en_tilte["title"].apply(translate_text_ensure)

/var/folders/br/6xtfsjcd1pld8d58t5wwkmcr0000gn/T/ipykernel_14856/1949120214.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_en_text["text_translated"] = df_non_en_text["text"].apply(translate_text_ensure)
/var/folders/br/6xtfsjcd1pld8d58t5wwkmcr0000gn/T/ipykernel_14856/1949120214.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_en_tilte["title_translated"] = df_non_en_tilte["title"].apply(translate_text_ensure)


In [8]:
print("Number of Rows with seperate translation than the original one: ", df_non_en_tilte[df_non_en_tilte['title'] != df_non_en_tilte['title_translated']].shape[0] , "Total Number of roWS non-en in column title : ", df_non_en_tilte.shape[0])
df_non_en_tilte[df_non_en_tilte['title'] != df_non_en_tilte['title_translated']] 

Number of Rows with seperate translation than the original one:  23 Total Number of roWS non-en in column title :  360


,year,month,day,country,title,text,hazard-category,product-category,hazard,product,language_text,language_title,title_translated
68,1999,9,7,au,Acquilla Bulk Trading—Sasso Pasta D'acciughe L...,PRA No. 1999/4071 Date published 7 Sep 1999 Pr...,chemical,seafood,toxin,anchovy paste,en,it,Acquilla Bulk Trading—Sasso Anchovy Paste Sea ...
100,2001,7,31,au,Pagasa—Dried Mackeral,PRA No. 2001/4849 Date published 31 Jul 2001 P...,chemical,seafood,toxin,mackerel,en,mt,Hope—Dried Mackeral
148,2003,1,23,au,Paesanella Cheese—Bocconcini cheese,PRA No. 2003/5936 Date published 23 Jan 2003 P...,biological,"meat, egg and dairy products",escherichia coli,cheese,en,it,Paesanella Cheese—Cheese Bites
174,2003,9,23,au,Barilla—Arrabbiata Pasta Sauce & Barilla Pesto...,PRA No. 2003/6478 Date published 23 Sep 2003 P...,chemical,"soups, broths, sauces and condiments",other,pasta sauce,en,it,Barilla—Arrabbiata Pasta Sauce & Barilla Calab...
261,2006,6,1,au,Tibaldi Smallgoods—Loin Capocollo mild,PRA No. 2006/8543 Date published 1 Jun 2006 Pr...,biological,"meat, egg and dairy products",salmonella,pork fillets,en,it,Tibaldi Smallgoods—Loin Capocollo Mild
350,2008,10,14,au,Bella Fantasia—240mL Hazelnut Gelato and 3-pac...,PRA No. 2008/10377 Date published 14 Oct 2008 ...,biological,ices and desserts,listeria monocytogenes,ice cream,en,it,Bella Fantasia—240mL Hazelnut Gelato and 3-pac...
442,2009,10,28,au,"Ettason Pty Ltd—Punchun Sesame Sauce, 290g jar",PRA No. 2009/11184 Date published 28 Oct 2009 ...,fraud,"nuts, nut products and seeds",labelling/misdescription,sesame,en,id,"Ettason Pty Ltd—Punch Sesame Sauce, 290g jar"
542,2011,5,9,au,Fetta - Microbial (E. coli) - 9 May 2011,Problem: Cottage Cheese Farm is conducting a r...,biological,"meat, egg and dairy products",escherichia coli,cheese,en,fi,Slice - Microbial (E. coli) - 9 May 2011
548,2011,6,6,au,Hunsa Smallgoods—Hunza Chorizo,PRA No. 2011/12628 Date published 6 Jun 2011 P...,biological,"meat, egg and dairy products",listeria monocytogenes,chorizo sausage,en,mt,Hunza Smallgoods—Hunza Chorizo
568,2011,9,17,us,2013 - Del Bueno Recalls Queso Fresco Casero C...,"Del Bueno of Grandview, WA is recalling all 16...",biological,"meat, egg and dairy products",listeria monocytogenes,fresh cheese,en,it,2013 - Del Bueno Recalls Queso Fresco Casero C...


- Key Note: Despite the Englidh translation in the column `title` we observe that only 24 out of 360 changed. 
- And the translations for these 23 rows is trivial due to product names mostly. 
- So, we will apply translation only for teh column "text" despite the low number of rows being equal to 3. 

In [9]:
df_food_trial = df_food_trial.drop(df_non_en_text.index) # Drop rows from original_df based on indexes in isolated_df
df_food_trial.reset_index(drop=True, inplace=True) # Reset the index if needed
df_food_trial

,year,month,day,country,title,text,hazard-category,product-category,hazard,product,language_text,language_title
0,1994,1,7,us,Recall Notification: FSIS-024-94,Case Number: 024-94 \n Date Opene...,biological,"meat, egg and dairy products",listeria monocytogenes,smoked sausage,en,en
1,1994,3,10,us,Recall Notification: FSIS-033-94,Case Number: 033-94 \n Date Opene...,biological,"meat, egg and dairy products",listeria spp,sausage,en,en
2,1994,3,28,us,Recall Notification: FSIS-014-94,Case Number: 014-94 \n Date Opene...,biological,"meat, egg and dairy products",listeria monocytogenes,ham slices,en,en
3,1994,4,3,us,Recall Notification: FSIS-009-94,Case Number: 009-94 \n Date Opene...,foreign bodies,"meat, egg and dairy products",plastic fragment,thermal processed pork meat,en,en
4,1994,7,1,us,Recall Notification: FSIS-001-94,Case Number: 001-94 \n Date Opene...,foreign bodies,"meat, egg and dairy products",plastic fragment,chicken breast,en,en
...,...,...,...,...,...,...,...,...,...,...,...,...
5074,2022,7,28,hk,Imported biscuit may contain allergen (peanuts),Imported biscuit may contain allergen (peanuts...,allergens,cereals and bakery products,peanuts and products thereof,biscuits,en,en
5075,2022,7,28,us,Danny’s Sub and Pizza Recalls Meat Pizza Produ...,023-2022\n\n \n High - Class I\n\n Produc...,fraud,prepared dishes and snacks,inspection issues,pizza,en,en
5076,2022,7,29,us,Lyons Magnus Voluntarily Recalls 53 Nutritiona...,"FRESNO, Calif. – July 28, 2022 – Lyons Magnus ...",biological,non-alcoholic beverages,cronobacter spp,non-alcoholic beverages,en,en
5077,2022,7,30,us,"Conagra Brands, Inc., Recalls Frozen Beef Prod...",025-2022\n\n \n High - Class I\n\n Misbra...,allergens,"meat, egg and dairy products",eggs and products thereof,frozen beef products,en,en


- We removed the three rows.

In [10]:
# before intagret translated text data to the main datfarame we will rename, drop related columns and order the columns follow the order of df_food_trial df.
df_non_en_text = df_non_en_text.drop(columns=["text"])
df_non_en_text = df_non_en_text.rename(columns={"text_translated": "text"})
desired_order = ["year", "month", "day","country", "title", "text", "hazard-category", "product-category", "hazard", "product", "language_text", "language_title"]
df_non_en_text = df_non_en_text[desired_order]
df_non_en_text

,year,month,day,country,title,text,hazard-category,product-category,hazard,product,language_text,language_title
868,2013,7,13,us,2009 - spanish: torres hillsdale country chees...,FOR IMMEDIATE RELEASE -- FOR IMMEDIATE ANNOUNC...,biological,"meat, egg and dairy products",listeria monocytogenes,cheese,es,en
4855,2021,3,3,us,El Abuelito EXPANDE SU RETIRO DEL MERCADO PARA...,"When a company announces a recall, withdrawal,...",biological,"meat, egg and dairy products",listeria monocytogenes,cheese,es,es
5093,2021,6,28,ca,"Lelièvre, Lelièvre & Lemoignan ltée. brand « H...","Notification - Lelièvre, Lelièvre & Lemoignan ...",biological,seafood,listeria monocytogenes,lobster,fr,fr


In [11]:
df_food_trial = pd.concat([df_food_trial, df_non_en_text], ignore_index=True)
df_food_trial

,year,month,day,country,title,text,hazard-category,product-category,hazard,product,language_text,language_title
0,1994,1,7,us,Recall Notification: FSIS-024-94,Case Number: 024-94 \n Date Opene...,biological,"meat, egg and dairy products",listeria monocytogenes,smoked sausage,en,en
1,1994,3,10,us,Recall Notification: FSIS-033-94,Case Number: 033-94 \n Date Opene...,biological,"meat, egg and dairy products",listeria spp,sausage,en,en
2,1994,3,28,us,Recall Notification: FSIS-014-94,Case Number: 014-94 \n Date Opene...,biological,"meat, egg and dairy products",listeria monocytogenes,ham slices,en,en
3,1994,4,3,us,Recall Notification: FSIS-009-94,Case Number: 009-94 \n Date Opene...,foreign bodies,"meat, egg and dairy products",plastic fragment,thermal processed pork meat,en,en
4,1994,7,1,us,Recall Notification: FSIS-001-94,Case Number: 001-94 \n Date Opene...,foreign bodies,"meat, egg and dairy products",plastic fragment,chicken breast,en,en
...,...,...,...,...,...,...,...,...,...,...,...,...
5077,2022,7,30,us,"Conagra Brands, Inc., Recalls Frozen Beef Prod...",025-2022\n\n \n High - Class I\n\n Misbra...,allergens,"meat, egg and dairy products",eggs and products thereof,frozen beef products,en,en
5078,2022,7,30,us,Rachael’s Food Corporation Recalls Ready-To-Ea...,024-2022\n\n \n High - Class I\n\n Produc...,biological,prepared dishes and snacks,listeria monocytogenes,wraps,en,en
5079,2013,7,13,us,2009 - spanish: torres hillsdale country chees...,FOR IMMEDIATE RELEASE -- FOR IMMEDIATE ANNOUNC...,biological,"meat, egg and dairy products",listeria monocytogenes,cheese,es,en
5080,2021,3,3,us,El Abuelito EXPANDE SU RETIRO DEL MERCADO PARA...,"When a company announces a recall, withdrawal,...",biological,"meat, egg and dairy products",listeria monocytogenes,cheese,es,es


In [12]:
# check fpor language; if our change passed.
df_food_trial['language_text_after_preprocess'] = df_food_trial['text'].apply(lambda x: langid.classify(x)[0])

- Let's check if the translation passes successfully. 

In [13]:
df_food_trial[df_food_trial['language_text_after_preprocess'] != 'en']

,year,month,day,country,title,text,hazard-category,product-category,hazard,product,language_text,language_title,language_text_after_preprocess
5081,2021,6,28,ca,"Lelièvre, Lelièvre & Lemoignan ltée. brand « H...","Notification - Lelièvre, Lelièvre & Lemoignan ...",biological,seafood,listeria monocytogenes,lobster,fr,fr,fr


In [14]:
df_food_trial['text'][df_food_trial['language_text_after_preprocess'] != 'en'].tolist()

['Notification - Lelièvre, Lelièvre & Lemoignan ltée. brand « Homards cuits » recalled due to Listeria monocytogenes From: Canadian Food Inspection Agency Recall / advisory date: June 28, 2021 Reason for recall / advisory: Microbiological - Listeria Hazard classification: Class 1 Company / Firm: Lelièvre, Lelièvre & Lemoignan ltée. Distribution: Quebec Extent of the distribution: Hotel/Restaurant/Institutional Reference number: 14473 Contents Affected products Public enquiries and media Affected products Brand Name Common Name Size UPC Code(s) on Product Lelièvre. Lelièvre & Lemoignan ltée. « Homards cuits » 30 lbs None 172-21 Packaging date: 21 juin 2021 Public enquiries and media Public enquiries Toll-free: 1-800-442-2342 (Canada and U.S.) Telephone: 1-613-773-2342 (local or international) Email: information@inspection.gc.ca Media relations Telephone: 613-773-6600 Email: cfia.media.acia@inspection.gc.ca Report a problem on this page Date modified: 2021-06-29']

- We will isolate in seperate dataframes teh row havinf non english text data in teh columns "title" and "text".
- We will apply the english translatin in the related columns leveraging teh function `translate_text_ensure`. 

- Maybe Google Translator confused as there are english terms, too. But, I will not insist on it more due to the low semantic impact.
- `Due to low semantic in the analysis, we decided to remove this approach`.